# Exercício de Fine Tuning do BERT com PyTorch
Neste exercício, você irá implementar o fine tuning do BERT adicionando Task Head com finalidade de classificação. Sugestões utilizando a biblioteca PyTorch.

Conforme visto em sala, o Self-attention é um componente central dos Transformers, as redes neurais que impulsionam os modelos de linguagem modernos como o BERT. Após as camadas de atenção dos modelos, podemos adicionar uma ou mais camadas com a finalidade de executar tarefas específicas.

## Contexto
Um modelo BERT é um Transformer do tipo Encoder que processa um texto de entrada gerando representações semânticas desse texto. O Self-attention permite que o modelo determine a relação entre diferentes tokens em uma sequência.

Adicionaremos, conforme visto em sala de aula, Task Head ao modelo para fazer fine tuning de classificação.



---




### [<img src="https://colab.google/static/images/icons/colab.png" width=100> OPCIONAL ] Configuração do ambiente para melhor desempenho e instalação de dependências
**Obs**: Selecione um ambiente com GPU para rodar esse notebook. No Google Colab:
**Runtime > Change runtime type > Hardware accelerator > GPU > GPU type > T4**.

---

In [1]:
# Caso esteja no Google Colab será necessário instalar apenas as dependências abaixo
!pip install seqeval>=1.2.2
!pip install evaluate>=0.4.0

# Importando dataset

Utilizaremos o dataset Rotten Tomatoes, que contém avaliações de filmes (https://huggingface.co/datasets/cornell-movie-review-data/rotten_tomatoes).

In [2]:
from datasets import load_dataset

# Preparando dados
tomatoes = load_dataset("rotten_tomatoes")
# Separando
train_data, test_data = tomatoes["train"], tomatoes["test"]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [3]:
# Vejamos o que tem aqui...
print(train_data)
print(train_data[:10])
print(train_data[-10:])

Dataset({
    features: ['text', 'label'],
    num_rows: 8530
})
{'text': ['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .', 'the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson\'s expanded vision of j . r . r . tolkien\'s middle-earth .', 'effective but too-tepid biopic', 'if you sometimes like to go to the movies to have fun , wasabi is a good place to start .', "emerges as something rare , an issue movie that's so honest and keenly observed that it doesn't feel like one .", 'the film provides some great insight into the neurotic mindset of all comics -- even those who have reached the absolute top of the game .', 'offers that rare combination of entertainment and education .', 'perhaps no picture ever made has more literally showed that 

# Carregando o modelo BERT

Utilizaremos checkpoint "bert-base-uncased".

Carregaremos o tokenizer, conforme exercícios anteriores, mas também a classe `AutoModelForSequenceClassification`.

A classe `AutoModelForSequenceClassification` da biblioteca Hugging Face Transformers é uma classe automática projetada para simplificar o carregamento de modelos de Sequence Classification (Classificação de Sequência) já criando nossa Task Head.

Vejamos a diferença entre o carregamento do modelo original e o modelo com a Task Head Classification.

In [4]:
from transformers import AutoModel, AutoModelForSequenceClassification, AutoConfig
import torch.nn as nn

# Suprime warnings
import warnings
warnings.filterwarnings("ignore")

# O checkpoint base do BERT (sem ajuste fino para tarefa específica)
MODEL_CHECKPOINT = "bert-base-uncased"
QNT_CLASSES = 2  # Definimos 2 classes (ex: positivo, negativo)

print("1. Carregando APENAS o Backbone BERT (BertModel)")
# AutoModel carrega o modelo base (o 'backbone' ou 'corpo' do transformer)
modelo_base = AutoModel.from_pretrained(MODEL_CHECKPOINT)

# Exibe a estrutura do modelo base
print(modelo_base)

print("\n" + "="*80 + "\n")

print(f"2. Carregando BERT com a Cabeça de Classificação ({QNT_CLASSES} classes)")

# 2.1. Criar uma configuração para 2 classes
config = AutoConfig.from_pretrained(MODEL_CHECKPOINT, num_labels=QNT_CLASSES)

# 2.2. Carregar o modelo usando AutoModelForSequenceClassification
modelo_class = AutoModelForSequenceClassification.from_pretrained(
    MODEL_CHECKPOINT,
    config=config
)

# Exibe a estrutura do modelo de Classificação
print(modelo_class)

1. Carregando APENAS o Backbone BERT (BertModel)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

## Carregando novamente o modelo para utilização em nossa prática

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

modelo = AutoModelForSequenceClassification.from_pretrained(MODEL_CHECKPOINT, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

## Tokenizando a entrada

In [6]:
from transformers import DataCollatorWithPadding

# Necessário para otimizar o padding no batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Definindo função de preprocessamento dos dados
def preprocessamento(itens):
   return tokenizer(itens["text"], truncation=True)

# Tokenizando dados train e test
tokenized_train = train_data.map(preprocessamento, batched=True)
tokenized_test = test_data.map(preprocessamento, batched=True)

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

## Definição de métricas

In [7]:
import numpy as np
import evaluate


def compute_metrics(eval_pred):
    """Calculate F1 score"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    load_f1 = evaluate.load("f1")
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"f1": f1}

# Treinamento

In [8]:
from transformers import TrainingArguments, Trainer

# Argumentos do treinamento
training_args = TrainingArguments(
   "model",
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=1,
   weight_decay=0.01,
   save_strategy="epoch",
   report_to="none"
)

# Instanciando o objeto "treinador"
treinador = Trainer(
   model=modelo,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)


In [9]:
treinador.train()

Step,Training Loss
500,0.388400


TrainOutput(global_step=534, training_loss=0.3847647016861019, metrics={'train_runtime': 25.7334, 'train_samples_per_second': 331.476, 'train_steps_per_second': 20.751, 'total_flos': 213940121334480.0, 'train_loss': 0.3847647016861019, 'epoch': 1.0})

## Avaliando os resultados obtidos

In [10]:
treinador.evaluate()

{'eval_loss': 0.3675398826599121,
 'eval_f1': 0.8485981308411215,
 'eval_runtime': 2.5013,
 'eval_samples_per_second': 426.181,
 'eval_steps_per_second': 26.786,
 'epoch': 1.0}

## ...mas o que foi treinado afinal de contas?

Vamos exibir a configuração dos parâmetros para entender o que está acontecendo.

In [11]:
# Exibindo detalhes das camadas
for name, param in modelo.named_parameters():
    print(f"Parameter: {name} ----- {param.requires_grad}")

Parameter: bert.embeddings.word_embeddings.weight ----- True
Parameter: bert.embeddings.position_embeddings.weight ----- True
Parameter: bert.embeddings.token_type_embeddings.weight ----- True
Parameter: bert.embeddings.LayerNorm.weight ----- True
Parameter: bert.embeddings.LayerNorm.bias ----- True
Parameter: bert.encoder.layer.0.attention.self.query.weight ----- True
Parameter: bert.encoder.layer.0.attention.self.query.bias ----- True
Parameter: bert.encoder.layer.0.attention.self.key.weight ----- True
Parameter: bert.encoder.layer.0.attention.self.key.bias ----- True
Parameter: bert.encoder.layer.0.attention.self.value.weight ----- True
Parameter: bert.encoder.layer.0.attention.self.value.bias ----- True
Parameter: bert.encoder.layer.0.attention.output.dense.weight ----- True
Parameter: bert.encoder.layer.0.attention.output.dense.bias ----- True
Parameter: bert.encoder.layer.0.attention.output.LayerNorm.weight ----- True
Parameter: bert.encoder.layer.0.attention.output.LayerNorm.bia

# Congelamento de Camadas

Uma tática muito comum para fine tuning é o congelamento de camadas, ou seja, bloquear a atualização dos pesos no treinamento da rede neural.

O PyTorch permite configurar explicitamente quais parâmetros podem ser atualizados. O atributo `requires_grad` é um boolean que controla se o parâmetro requer cálculo de gradiente. Ou seja, quando `requires_grad == False` o sistema de autograd do PyTorch não rastreia as operações que o envolvem, não calcula nem armazena seu gradiente durante o backpropagation, e, consequentemente, o otimizador não ajusta seu valor, mantendo-o constante ao longo de todo o processo de treinamento.

---

Vamos carregar novamente o modelo, mas agora vamos ajustar essa flag apenas nos parâmetros da camada de classificação.



In [12]:
# Load Model and Tokenizer
modelo_bert_congelado = AutoModelForSequenceClassification.from_pretrained(MODEL_CHECKPOINT, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

for name, param in modelo_bert_congelado.named_parameters():
    # Ajusta a camada "classifier"
    if name.startswith("classifier"):
      param.requires_grad = True
    else:
      param.requires_grad = False
    print(f"Parameter: {name} ----- {param.requires_grad}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Parameter: bert.embeddings.word_embeddings.weight ----- False
Parameter: bert.embeddings.position_embeddings.weight ----- False
Parameter: bert.embeddings.token_type_embeddings.weight ----- False
Parameter: bert.embeddings.LayerNorm.weight ----- False
Parameter: bert.embeddings.LayerNorm.bias ----- False
Parameter: bert.encoder.layer.0.attention.self.query.weight ----- False
Parameter: bert.encoder.layer.0.attention.self.query.bias ----- False
Parameter: bert.encoder.layer.0.attention.self.key.weight ----- False
Parameter: bert.encoder.layer.0.attention.self.key.bias ----- False
Parameter: bert.encoder.layer.0.attention.self.value.weight ----- False
Parameter: bert.encoder.layer.0.attention.self.value.bias ----- False
Parameter: bert.encoder.layer.0.attention.output.dense.weight ----- False
Parameter: bert.encoder.layer.0.attention.output.dense.bias ----- False
Parameter: bert.encoder.layer.0.attention.output.LayerNorm.weight ----- False
Parameter: bert.encoder.layer.0.attention.output

## Treinando o modelo carregado novamente

Agora será executado o processo de treinamento e vejamos a diferença.

In [13]:
from transformers import TrainingArguments, Trainer

treinador = Trainer(
   model=modelo_bert_congelado,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

treinador.train()

Step,Training Loss
500,0.694800


TrainOutput(global_step=534, training_loss=0.6949845181868763, metrics={'train_runtime': 10.6644, 'train_samples_per_second': 799.86, 'train_steps_per_second': 50.073, 'total_flos': 213940121334480.0, 'train_loss': 0.6949845181868763, 'epoch': 1.0})

## Nova Avaliação do treinamento

In [14]:
treinador.evaluate()

{'eval_loss': 0.6907061338424683,
 'eval_f1': 0.6344005956813105,
 'eval_runtime': 2.209,
 'eval_samples_per_second': 482.563,
 'eval_steps_per_second': 30.33,
 'epoch': 1.0}

In [15]:
from datasets import concatenate_datasets

# Pegar exemplos do dataset de teste (mistura de positivos e negativos)
positive_reviews = test_data.filter(lambda example: example["label"] == 1).select(range(5))
negative_reviews = test_data.filter(lambda example: example["label"] == 0).select(range(5))

# Combinar os exemplos
sample_reviews = concatenate_datasets([positive_reviews, negative_reviews])


# Preprocessar as avaliações individualmente
tokenized_sample_list = [tokenizer(review, truncation=True) for review in sample_reviews['text']]

# Usar o data collator para preparar a entrada para o modelo
import torch
batch = data_collator(tokenized_sample_list)

# Mover o batch para o mesmo dispositivo do modelo
batch = {k: v.to(modelo_bert_congelado.device) for k, v in batch.items()}


# Fazer previsões
with torch.no_grad():
    outputs = modelo_bert_congelado(**batch)
    predictions = torch.argmax(outputs.logits, dim=-1)

# Exibir os resultados
for i, review in enumerate(sample_reviews["text"]):
    predicted_label = "Positive" if predictions[i].item() == 1 else "Negative"
    # label real para fins de comparação
    actual_label = "Positive" if sample_reviews['label'][i] == 1 else "Negative"
    print(f"Review: {review}\nPredicted Label: {predicted_label}\nActual Label: {actual_label}\n")

Filter:   0%|          | 0/1066 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1066 [00:00<?, ? examples/s]

Review: lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .
Predicted Label: Positive
Actual Label: Positive

Review: consistently clever and suspenseful .
Predicted Label: Positive
Actual Label: Positive

Review: it's like a " big chill " reunion of the baader-meinhof gang , only these guys are more harmless pranksters than political activists .
Predicted Label: Positive
Actual Label: Positive

Review: the story gives ample opportunity for large-scale action and suspense , which director shekhar kapur supplies with tremendous skill .
Predicted Label: Positive
Actual Label: Positive

Review: red dragon " never cuts corners .
Predicted Label: Positive
Actual Label: Positive

Review: this slender plot feels especially thin stretched over the nearly 80-minute running time .
Predicted Label: Negative
Actual Label: Negative

Review: a film that will probably please people already fascinated by behan but leave e

## Para testar o classificador com uma string sua, utilize o código abaixo:

Lembre-se de enviar a variável para a GPU, para não ver nenhum erro bizarro.

In [16]:
texto_teste = "This is a horrible movie"

# Executa inferência no modelo treinado
# Mudando para o modo eval
modelo.eval()

entrada_tokenizada = tokenizer(texto_teste, return_tensors="pt")

# Move os tensores para o dispositivo em que o modelo se encontra
device = modelo.device
entrada_tokenizada = {k: v.to(device) for k, v in entrada_tokenizada.items()}

with torch.no_grad():
  saida = modelo_bert_congelado(**entrada_tokenizada)

print(f"Saída bruta: {saida}")

# Trabalhando a saída bruta
logits = saida.logits

prob = torch.softmax(logits, dim=1)
print(f"Probabilidades: {prob}")

Saída bruta: SequenceClassifierOutput(loss=None, logits=tensor([[0.4323, 0.2332]], device='cuda:0'), hidden_states=None, attentions=None)
Probabilidades: tensor([[0.5496, 0.4504]], device='cuda:0')



---

## Tarefas do Exercício

1. Agora responda com suas palavras o que aconteceu em ambos os casos durante o treinamento, evidenciando se você percebeu alguma diferença durante o passo de treinamento. Discuta brevemente os resultados obtidos.

R: No primeiro caso de treinamento, utilizou-se a abordagem Full Fine-Tuning, isto é, todas as camadas estavam descongeladas. Obtendo-se um tempo de treino de 97.2 segundos e um F1-Score de 0.848.

Já no segundo caso, utilizou-se a abordagem Fine-Tuning com Congelamento, isto é, todas as camadas estavam congeladas, com excessão da cabeça de classificação. Obtendo-se um tempo de treinamento de 49.8 segundos e um F1-Score de 0.639.

Como no segundo caso ocorreu o congelamento de camadas, existiam menos parâmetros para atualização, o que resultou em um menor tempo de treino. Em contrapartida, a performance foi inferior, devido a dificuldade do modelo se adaptar ao novo domínio.

Basicamente, ao fazer um Fine-Tuning com Congelamento, existe certa dificuldade de adaptar o modelo pré-existente (backbone) à nova camada treinada (cabeça de classificação).

O BERT pré-treinado em dados genéricos cria embeddings e representações de texto que são bons para tarefas gerais. No entanto, o presente dataset tem um vocabulário próprio. O Full Fine-Tuning permite que todas as 12 camadas do BERT ajustem 100% dos parâmetros para criar representações que são otimizadas para o domínio específico.

2. Conforme instruções anteriores, carregue mais uma vez o modelo BERT, mas agora mantenha apenas a partir da camada encoder 10 (`bert.encoder.layer.10`) do modelo BERT como treinável (`require_grad == True`). Ou seja, congele (`requires_grad == False`) até a camada encoder 9 (`bert.encoder.layer.9`).

In [17]:
# Load Model and Tokenizer
modelo_bert_parcialmente_congelado = AutoModelForSequenceClassification.from_pretrained(MODEL_CHECKPOINT, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

for name, param in modelo_bert_parcialmente_congelado.named_parameters():
    # Ajusta a camada "encoder"
    encoder_id = name.split(".")
    if len(encoder_id) > 3 and encoder_id[1] == "encoder":
        encoder_id = int(encoder_id[3])

        if encoder_id >= 10:
            param.requires_grad = True
        else:
            param.requires_grad = False

    # Ajusta a camada "pooler"
    elif name.startswith("bert.pooler"):
        param.requires_grad = True

    # Ajusta a camada "classifier"
    elif name.startswith("classifier"):
      param.requires_grad = True

    else:
      param.requires_grad = False

    print(f"Parameter: {name} ----- {param.requires_grad}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Parameter: bert.embeddings.word_embeddings.weight ----- False
Parameter: bert.embeddings.position_embeddings.weight ----- False
Parameter: bert.embeddings.token_type_embeddings.weight ----- False
Parameter: bert.embeddings.LayerNorm.weight ----- False
Parameter: bert.embeddings.LayerNorm.bias ----- False
Parameter: bert.encoder.layer.0.attention.self.query.weight ----- False
Parameter: bert.encoder.layer.0.attention.self.query.bias ----- False
Parameter: bert.encoder.layer.0.attention.self.key.weight ----- False
Parameter: bert.encoder.layer.0.attention.self.key.bias ----- False
Parameter: bert.encoder.layer.0.attention.self.value.weight ----- False
Parameter: bert.encoder.layer.0.attention.self.value.bias ----- False
Parameter: bert.encoder.layer.0.attention.output.dense.weight ----- False
Parameter: bert.encoder.layer.0.attention.output.dense.bias ----- False
Parameter: bert.encoder.layer.0.attention.output.LayerNorm.weight ----- False
Parameter: bert.encoder.layer.0.attention.output

3. Execute o treinamento com os mesmos parâmetros utilizados anteriormente e execute o método `evaluate()` para calcular as métricas e discuta os resultados obtidos.
Caso julgar necessário, execute outros treinamentos ajustando quantidades distintas de parâmetros treináveis para tirar conclusões adicionais.

In [18]:
treinador_congelamento_parcial = Trainer(
   model=modelo_bert_parcialmente_congelado,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

treinador_congelamento_parcial.train()

Step,Training Loss
500,0.450900


TrainOutput(global_step=534, training_loss=0.44675309470530306, metrics={'train_runtime': 12.7269, 'train_samples_per_second': 670.231, 'train_steps_per_second': 41.958, 'total_flos': 213940121334480.0, 'train_loss': 0.44675309470530306, 'epoch': 1.0})

In [19]:
treinador_congelamento_parcial.evaluate()

{'eval_loss': 0.41342198848724365,
 'eval_f1': 0.82277318640955,
 'eval_runtime': 2.1907,
 'eval_samples_per_second': 486.601,
 'eval_steps_per_second': 30.584,
 'epoch': 1.0}

Para esse último modelo, a melhor combinação de resultados foi atingindo até então. Ele apresentou o tempo de treino muito próximo ao Fine-Tuning com Congelamento (apenas a cabeça de classificação descongelada) e performance muito próximo ao Full Fine-Tuning.

4. (BÔNUS) Pesquise e escolha outro dataset para fazer fine tuning do modelo BERT.

In [20]:
# Importando Dataset

agn = load_dataset("ag_news")
agn_train_data, agn_test_data = agn["train"], agn["test"]

print(agn_train_data)
print(agn_train_data[0])
print(agn_train_data[-10:])

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 120000
})
{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'label': 2}
{'text': ['Barack Obama Gets  #36;1.9 Million Book Deal (AP) AP - U.S. Sen.-elect Barack Obama, whose 1995 book jumped onto best seller lists after his keynote address to the Democratic National Convention, has landed a three-book deal worth  #36;1.9 million.', 'Rauffer Beats Favorites to Win Downhill  VAL GARDENA, Italy (Reuters) - Max Rauffer became the first  German man in nearly 13 years to win an Alpine ski World Cup  race when he beat the favorites in a wind-affected downhill on  Saturday.', 'Iraqis Face Winter Shivering by Candlelight  BAGHDAD (Reuters) - As if the daily struggle to dodge  bullets and bombings is not enough, many Iraqis now face a  freezing winter shivering by candlelight as persistent attacks  keep the power out for more than 12

In [21]:
# Tokenizando a Entrada

labels = agn_train_data.features["label"].names
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}
NUM_LABELS = len(labels)

print(f"Mapeamento de IDs para Rótulos: {id2label}")

# Criamos uma função para tokenizar o texto de todo o dataset

def tokenize_function(examples):
    """Aplica o tokenizador ao campo 'text' dos exemplos."""
    return tokenizer(examples["text"], truncation=True, max_length=512)

tokenized_train = agn_train_data.map(tokenize_function, batched=True)
tokenized_eval = agn_test_data.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Mapeamento de IDs para Rótulos: {0: 'World', 1: 'Sports', 2: 'Business', 3: 'Sci/Tech'}


Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [22]:
# Carregando o Modelo BERT

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_CHECKPOINT,
    num_labels=NUM_LABELS,
    id2label=id2label,
    label2id=label2id
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
# Definição de Métricas

from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    """Calcula acurácia e F1-score para a avaliação."""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = accuracy_score(labels, predictions)
    # Usamos 'weighted' para o F1-score por ser uma tarefa multi-classe
    f1 = f1_score(labels, predictions, average="weighted")

    return {"accuracy": accuracy, "f1": f1}

In [24]:
# Treinamento

training_args = TrainingArguments(
    output_dir="./ag_news_bert_finetuned", # Diretório para salvar o modelo
    learning_rate=2e-5,                   # Taxa de aprendizado
    per_device_train_batch_size=16,       # Tamanho do batch de treino
    per_device_eval_batch_size=16,        # Tamanho do batch de avaliação
    num_train_epochs=3,                   # Número de épocas (3 costuma ser um bom valor para fine-tuning)
    weight_decay=0.01,                    # Regularização
   save_strategy="epoch",
   report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Step,Training Loss
500,0.406900
1000,0.258200
1500,0.231100
2000,0.225800
2500,0.231600
3000,0.237500
3500,0.206100
4000,0.209300
4500,0.204900
5000,0.197300


TrainOutput(global_step=22500, training_loss=0.1483897198147244, metrics={'train_runtime': 1555.5336, 'train_samples_per_second': 231.432, 'train_steps_per_second': 14.464, 'total_flos': 1.7987934973367424e+16, 'train_loss': 0.1483897198147244, 'epoch': 3.0})

In [25]:
# Avaliação dos Resultados

eval_results = trainer.evaluate(eval_dataset=tokenized_eval)

In [28]:
print(eval_results)

{'eval_loss': 0.2335948348045349, 'eval_accuracy': 0.9471052631578948, 'eval_f1': 0.9471169240933109, 'eval_runtime': 11.1588, 'eval_samples_per_second': 681.079, 'eval_steps_per_second': 42.567, 'epoch': 3.0}


In [26]:
trainer.save_model("./ag_news_bert_finetuned/best_model")

In [27]:
# Teste de Classificação

def classify_text(text):
    """
    Recebe um texto (string), tokeniza, faz a predição com o modelo
    e retorna o rótulo da classe prevista.
    """
    # Coloca o modelo em modo de avaliação
    model.eval()

    # Tokeniza o texto de entrada
    # return_tensors="pt" formata a saída como tensores do PyTorch
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)

    # Move os dados tokenizados para o mesmo dispositivo do modelo (GPU/CPU)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    # Desativa o cálculo de gradientes para acelerar a inferência
    with torch.no_grad():
        outputs = model(**inputs)

    # As saídas do modelo são 'logits' (valores brutos antes da ativação softmax)
    logits = outputs.logits

    # Encontra o índice da classe com o maior logit
    predicted_class_id = torch.argmax(logits, dim=1).item()

    # Usa o mapeamento 'id2label' do modelo para obter o nome da classe
    return model.config.id2label[predicted_class_id]


# Exemplo 1: Ciência/Tecnologia
text_scitech = "Researchers have developed a new type of battery using sodium-ion technology, which could dramatically reduce the cost of electric vehicles and energy storage systems."
predicted_category = classify_text(text_scitech)
print(f"Texto: '{text_scitech}'")
print(f"Categoria Prevista: {predicted_category}\n") # Esperado: Sci/Tech

# Exemplo 2: Esportes
text_sports = "The final match of the championship will be held next Sunday between the two rival teams after a thrilling semi-final that went into extra time."
predicted_category = classify_text(text_sports)
print(f"Texto: '{text_sports}'")
print(f"Categoria Prevista: {predicted_category}\n") # Esperado: Sports

# Exemplo 3: Negócios
text_business = "The stock market reacted positively to the central bank's announcement to hold interest rates steady, with the technology sector seeing the biggest gains."
predicted_category = classify_text(text_business)
print(f"Texto: '{text_business}'")
print(f"Categoria Prevista: {predicted_category}\n") # Esperado: Business

# Exemplo 4: Mundo
text_world = "Diplomats from several nations met in Geneva today to discuss a new treaty on international trade and climate change cooperation."
predicted_category = classify_text(text_world)
print(f"Texto: '{text_world}'")
print(f"Categoria Prevista: {predicted_category}\n") # Esperado: World

Texto: 'Researchers have developed a new type of battery using sodium-ion technology, which could dramatically reduce the cost of electric vehicles and energy storage systems.'
Categoria Prevista: Sci/Tech

Texto: 'The final match of the championship will be held next Sunday between the two rival teams after a thrilling semi-final that went into extra time.'
Categoria Prevista: Sports

Texto: 'The stock market reacted positively to the central bank's announcement to hold interest rates steady, with the technology sector seeing the biggest gains.'
Categoria Prevista: Business

Texto: 'Diplomats from several nations met in Geneva today to discuss a new treaty on international trade and climate change cooperation.'
Categoria Prevista: World

